In [1]:
import pandas as pd
import numpy as np # We'll likely need numpy for NaN and other operations

# Load the raw dataset
try:
    df_raw = pd.read_csv('fbref_2024_2025_pl_standard_stats_raw.csv')
    print("Successfully loaded fbref_2024_2025_pl_standard_stats_raw.csv")
    print(f"Initial raw shape: {df_raw.shape}")
except FileNotFoundError:
    print("Error: fbref_2024_2025_pl_standard_stats_raw.csv not found. Make sure it's in the same directory as the notebook.")
    df_raw = pd.DataFrame() # Create an empty DataFrame to avoid further errors if file not found

# Display the first few rows and basic info to start
if not df_raw.empty:
    print("\nFirst 5 rows of raw data:")
    print(df_raw.head())
    print("\nInfo about raw data types and non-null values:")
    df_raw.info()

Successfully loaded fbref_2024_2025_pl_standard_stats_raw.csv
Initial raw shape: (591, 37)

First 5 rows of raw data:
  Rk             Player   Nation    Pos        Squad     Age  Born  \
0  1         Max Aarons  eng ENG     DF  Bournemouth  25-132  2000   
1  2  Joshua Acheampong  eng ENG     DF      Chelsea  19-011  2006   
2  3        Tyler Adams   us USA     MF  Bournemouth  26-091  1999   
3  4   Tosin Adarabioyo  eng ENG     DF      Chelsea  27-234  1997   
4  5      Simon Adingra   ci CIV  FW,MF     Brighton  23-135  2002   

  Playing_Time_MP Playing_Time_Starts Playing_Time_Min  ...  \
0               3                   1               86  ...   
1               4                   2              170  ...   
2              26                  19             1785  ...   
3              20                  13             1229  ...   
4              27                  10              988  ...   

  Per_90_Minutes_Ast Per_90_Minutes_G+A Per_90_Minutes_G_PK  \
0               0.0

In [2]:
# Create a copy to work on
df = df_raw.copy()

# Identify rows that are likely headers
# A common characteristic of these header rows is that the 'Rk' column contains the string 'Rk'
# or another typically numeric column contains a non-numeric string that matches its header name.
header_row_mask = df['Rk'] == 'Rk' # Check where 'Rk' column has the string 'Rk'
num_header_rows = header_row_mask.sum()
print(f"Found {num_header_rows} potential repeated header rows based on 'Rk' column.")

# Display these potential header rows to confirm
if num_header_rows > 0:
    print("\nPotential header rows found:")
    print(df[header_row_mask])

# Remove these header rows
df = df[~header_row_mask] # Keep rows where the mask is False
print(f"\nShape after removing header rows: {df.shape}")

# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

print("\nFirst 5 rows after removing headers:")
print(df.head())
print("\nLast 5 rows after removing headers:")
print(df.tail())

Found 22 potential repeated header rows based on 'Rk' column.

Potential header rows found:
     Rk  Player  Nation  Pos  Squad  Age  Born Playing_Time_MP  \
25   Rk  Player  Nation  Pos  Squad  Age  Born              MP   
51   Rk  Player  Nation  Pos  Squad  Age  Born              MP   
77   Rk  Player  Nation  Pos  Squad  Age  Born              MP   
103  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
129  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
155  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
181  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
207  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
233  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
259  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
285  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
311  Rk  Player  Nation  Pos  Squad  Age  Born              MP   
337  Rk  Player  Nation  Pos  Squad  Age  Born    

In [3]:
# Investigate the 'Matches' column
print("Unique values in 'Matches' column and their counts:")
print(df['Matches'].value_counts(dropna=False)) # dropna=False will also show NaN counts if any

# If it's always 'Matches' or mostly 'Matches' and some NaNs, it's probably safe to drop.
# Let's assume for now it's not useful. We can decide to drop it later.

Unique values in 'Matches' column and their counts:
Matches
Matches    569
Name: count, dtype: int64


In [4]:
# Clean the 'Age' column to extract only the age in years
# The lambda function splits the string by '-' and takes the first part (the age)
# We'll convert to numeric later, after handling potential NaNs more broadly
df['Age_Clean'] = df['Age'].astype(str).apply(lambda x: x.split('-')[0] if pd.notnull(x) and '-' in x else x)

print("\n'Age' column before and after cleaning (first 5 rows):")
print(df[['Age', 'Age_Clean']].head())

# We can drop the original 'Age' column later if we're happy with 'Age_Clean'


'Age' column before and after cleaning (first 5 rows):
      Age Age_Clean
0  25-132        25
1  19-011        19
2  26-091        26
3  27-234        27
4  23-135        23


In [5]:
# List of columns that should be numeric
# Based on your column list and typical FBref data:
# 'Rk', 'Born', 'Playing_Time_MP', 'Playing_Time_Starts', 'Playing_Time_Min', 
# 'Playing_Time_90s', 'Performance_Gls', 'Performance_Ast', 'Performance_G+A', 
# 'Performance_G_PK', 'Performance_PK', 'Performance_PKatt', 'Performance_CrdY', 
# 'Performance_CrdR', 'Expected_xG', 'Expected_npxG', 'Expected_xAG', 
# 'Expected_npxG+xAG', 'Progression_PrgC', 'Progression_PrgP', 'Progression_PrgR',
# 'Per_90_Minutes_Gls', 'Per_90_Minutes_Ast', 'Per_90_Minutes_G+A', 
# 'Per_90_Minutes_G_PK', 'Per_90_Minutes_G+A_PK', 'Per_90_Minutes_xG', 
# 'Per_90_Minutes_xAG', 'Per_90_Minutes_xG+xAG', 'Per_90_Minutes_npxG', 
# 'Per_90_Minutes_npxG+xAG'
# Also our new 'Age_Clean'

# Let's define these columns explicitly
cols_to_convert_to_numeric = [
    'Rk', 'Born', 'Playing_Time_MP', 'Playing_Time_Starts', 'Playing_Time_Min', 
    'Playing_Time_90s', 'Performance_Gls', 'Performance_Ast', 'Performance_G+A', 
    'Performance_G_PK', 'Performance_PK', 'Performance_PKatt', 'Performance_CrdY', 
    'Performance_CrdR', 'Expected_xG', 'Expected_npxG', 'Expected_xAG', 
    'Expected_npxG+xAG', 'Progression_PrgC', 'Progression_PrgP', 'Progression_PrgR',
    'Per_90_Minutes_Gls', 'Per_90_Minutes_Ast', 'Per_90_Minutes_G+A', 
    'Per_90_Minutes_G_PK', 'Per_90_Minutes_G+A_PK', 'Per_90_Minutes_xG', 
    'Per_90_Minutes_xAG', 'Per_90_Minutes_xG+xAG', 'Per_90_Minutes_npxG', 
    'Per_90_Minutes_npxG+xAG', 'Age_Clean' # Add our cleaned Age column
]

# Convert these columns to numeric, coercing errors to NaN
for col in cols_to_convert_to_numeric:
    if col in df.columns: # Check if column exists before trying to convert
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else:
        print(f"Warning: Column '{col}' not found in DataFrame for numeric conversion.")

print("\nData types after attempting numeric conversion:")
df.info()

# Check for NaNs introduced by 'coerce' in a few key numeric columns
print("\nNaN counts in some key numeric columns after conversion:")
key_numeric_cols_check = ['Age_Clean', 'Playing_Time_Min', 'Performance_Gls', 'Expected_xG']
for col in key_numeric_cols_check:
    if col in df.columns:
        print(f"NaNs in {col}: {df[col].isnull().sum()}")


Data types after attempting numeric conversion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Rk                       569 non-null    int64  
 1   Player                   569 non-null    object 
 2   Nation                   566 non-null    object 
 3   Pos                      569 non-null    object 
 4   Squad                    569 non-null    object 
 5   Age                      566 non-null    object 
 6   Born                     566 non-null    float64
 7   Playing_Time_MP          569 non-null    int64  
 8   Playing_Time_Starts      569 non-null    int64  
 9   Playing_Time_Min         569 non-null    int64  
 10  Playing_Time_90s         569 non-null    float64
 11  Performance_Gls          569 non-null    int64  
 12  Performance_Ast          569 non-null    int64  
 13  Performance_G+A          569 no

In [6]:
# Drop the original 'Age' column as we have 'Age_Clean'
if 'Age' in df.columns:
    df.drop('Age', axis=1, inplace=True)
    print("Dropped original 'Age' column.")

# Drop the 'Matches' column as it's redundant
if 'Matches' in df.columns:
    df.drop('Matches', axis=1, inplace=True)
    print("Dropped 'Matches' column.")

# Rename 'Age_Clean' to 'Age' for simplicity
if 'Age_Clean' in df.columns:
    df.rename(columns={'Age_Clean': 'Age'}, inplace=True)
    print("Renamed 'Age_Clean' to 'Age'.")

print("\nDataFrame info after dropping columns and renaming 'Age_Clean':")
df.info()

# Let's look at the rows with NaN Age/Born
print("\nRows with missing 'Age' or 'Born':")
missing_age_born_rows = df[df['Age'].isnull() | df['Born'].isnull()]
print(missing_age_born_rows[['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born']])

# Decision on NaNs in 'Age' and 'Born':
# For 'Age' and 'Born', having 3 missing values out of ~570 players is very minor.
# Options:
# 1. Drop these 3 rows: If age/birth year is critical for all analysis.
# 2. Fill with a placeholder (e.g., mean, median, or a special value like 0 or -1): If we want to keep the players for other stats.
#    Mean/median imputation for age/birth year can be problematic as it introduces artificial data.
# 3. Leave as NaN: Many pandas operations and plotting libraries can handle NaNs gracefully (e.g., by ignoring them).

# For now, let's leave them as NaN. Most analyses will handle this correctly.
# If a specific analysis *requires* no NaNs in Age/Born, we can address it then.

# What about other NaNs?
# We saw 'Nation' has 3 NaNs (566 non-null out of 569 initially). These are likely the same 3 rows.
# We'll leave 'Nation' NaNs as is for now as well.

# Let's check for any other columns that might have all NaNs (unlikely now, but good check)
all_nan_cols = df.columns[df.isnull().all()].tolist()
if all_nan_cols:
    print(f"\nWarning: Columns with all NaN values: {all_nan_cols}. Consider dropping them.")
else:
    print("\nNo columns found with all NaN values.")


print("\nFinal check of first 5 rows with cleaned data structure:")
print(df.head())

Dropped original 'Age' column.
Dropped 'Matches' column.
Renamed 'Age_Clean' to 'Age'.

DataFrame info after dropping columns and renaming 'Age_Clean':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Rk                       569 non-null    int64  
 1   Player                   569 non-null    object 
 2   Nation                   566 non-null    object 
 3   Pos                      569 non-null    object 
 4   Squad                    569 non-null    object 
 5   Born                     566 non-null    float64
 6   Playing_Time_MP          569 non-null    int64  
 7   Playing_Time_Starts      569 non-null    int64  
 8   Playing_Time_Min         569 non-null    int64  
 9   Playing_Time_90s         569 non-null    float64
 10  Performance_Gls          569 non-null    int64  
 11  Performance_Ast          569 non-nul

In [7]:
# Save the cleaned DataFrame to a new CSV file
cleaned_csv_path = 'fbref_2024_2025_pl_standard_stats_cleaned.csv'
df.to_csv(cleaned_csv_path, index=False)

print(f"Cleaned player stats DataFrame saved to {cleaned_csv_path}")
print(f"Shape of cleaned DataFrame: {df.shape}")
print("First 5 rows of the final cleaned DataFrame:")
print(df.head())

Cleaned player stats DataFrame saved to fbref_2024_2025_pl_standard_stats_cleaned.csv
Shape of cleaned DataFrame: (569, 36)
First 5 rows of the final cleaned DataFrame:
   Rk             Player   Nation    Pos        Squad    Born  \
0   1         Max Aarons  eng ENG     DF  Bournemouth  2000.0   
1   2  Joshua Acheampong  eng ENG     DF      Chelsea  2006.0   
2   3        Tyler Adams   us USA     MF  Bournemouth  1999.0   
3   4   Tosin Adarabioyo  eng ENG     DF      Chelsea  1997.0   
4   5      Simon Adingra   ci CIV  FW,MF     Brighton  2002.0   

   Playing_Time_MP  Playing_Time_Starts  Playing_Time_Min  Playing_Time_90s  \
0                3                    1                86               1.0   
1                4                    2               170               1.9   
2               26                   19              1785              19.8   
3               20                   13              1229              13.7   
4               27                   10      